In [1]:
from datetime import datetime, timedelta
from heater_simulation import HeaterSimulation
from pid import PidDuty
import numpy as np
import pandas as pd
import altair as alt


def make_chart(d):
    data = pd.DataFrame(data={"time": np.concatenate([d["times"], d["times"], d["times"]], axis=0),
                              "value": np.concatenate([d["valve_states"],
                                                       d["liquid_temperatures"],
                                                       d["room_temperatures"]], axis=0),
                              "label": ["valve"] * len(times) + ["water"] * len(times) + ["temperature"] * len(times)})
    chart = alt.Chart(data).mark_line().encode(
        x='time',
        y='value',
        color='label',
        strokeDash='label')
    return chart


h_sim = HeaterSimulation()
controller = PidDuty()
times = np.arange(start=0.5, stop=10*60, step=0.5, dtype=float)
valve_states = np.empty_like(times)
liquid_temperatures = np.empty_like(times)
room_temperatures = np.empty_like(times)

In [90]:
h_sim.room_temperature = 17.0
h_sim.liquid_temperature = 17.0
controller.set_point(20.0)
controller.set_kp(0.25)
controller.set_ki(0.000001)
controller.set_kd(100000.0)
now = datetime.now()
h_sim.last_time = now
controller.last_time = now
for i, seconds in enumerate(times):
    time = now + timedelta(seconds=seconds)
    valve = controller.control_loop(measurement=h_sim.room_temperature, time=time)
    h_sim.simulation_step(flow_factor=valve, time=time)
    valve_states[i] = valve
    liquid_temperatures[i] = h_sim.liquid_temperature
    room_temperatures[i] = h_sim.room_temperature
print(room_temperatures[-1])
make_chart({"times": times,
            "valve_states": valve_states,
            "liquid_temperatures": liquid_temperatures,
            "room_temperatures": room_temperatures}).properties(width=750)

19.8592807990815


alt.Chart(...)

In [91]:
print(valve_states)
print(liquid_temperatures)
print(room_temperatures)

[1.         1.         1.         ... 0.0919859  0.1013358  0.08660418]
[17.01219376 17.03654824 17.0729997  ... 20.65774304 20.69101732
 20.63592659]
[16.99976037 16.99928927 16.99860101 ... 19.86719945 19.88087969
 19.8592808 ]
